# 데이터분석 test notebook

In [1]:
# !pip install seaborn
# !pip install tqdm
# !pip install -U scikit-learn
# !pip install emoji
# !pip install pyarrow
# !pip install fastparquet
# !pip install squarify
# !pip install wordcloud
# !pip install textblob

In [2]:
# library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import re
import squarify
from tqdm import tqdm, tqdm_pandas
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma, Hannanum
from collections import Counter
from emoji import core
from wordcloud import WordCloud
from matplotlib import font_manager, rc
from nltk import ngrams
from textblob import TextBlob

font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Kkma = Kkma()
hannanum = Hannanum()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
milk_t = pd.read_parquet('./data/test.parquet')
df = milk_t.copy()
df

,keyword_dict,contents,recommend,아이스크림,엘리하이
0,{'밀크': [' 밀크T는 로그인이 홈런보다는복잡한 편인데요 로그인학습기를 켜면 밀...,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False,True,True
1,{'밀크': [' 아하밀크T는 하단에 오늘의 퀴즈 오늘의 한자가 따로 있어서 함께 ...,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True,True,True
2,{'밀크': [' 이렇게 신청한 스마트 패드가5가지 교원 빨간펜 아이캔두 초등 천재...,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False,True,True
3,{'밀크': [' 전업주부니까 그게 저의 직업이라고 생각했어요하지만 모든걸잘하고 싶...,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False,True,True
4,{'밀크': [' 구분엘리하이7일윙크6일밀크T10일아이스크림 홈럼10일업체메가스터디...,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True,True,True
...,...,...,...,...,...
1006,{'밀크': [' 밀크티는 정액제라 대부분의 강의를 담아서 공부할있어요 물론 너무 ...,중학 스마트학습 천재교육 밀크티로. 온라인 학습하다가학교 다니고 하다 보니아이들이 ...,False,False,False
1007,{'밀크': [' ㅜㅜ조금고학년이면 좋아할있을지 모르겠지만현재엘리 하이 아이스크림 ...,홈쇼핑에서 2월 말부터 방송이 나오더라고요와이즈캠프도 있었는데 그건 신청을했어요우선...,False,True,True
1008,{'밀크': [' 그런데 밀크티에 있는 영상은 추가로 보여주거든요이 영상은 오늘의 ...,작년 이맘때 초등준비를 위해7세 홈스쿨로 무엇이 좋을까 고민 중이었어요. 학습적인 ...,False,False,False
1009,{'밀크': [' 밀크티는 온라인강의지만11 전담 관리선생님이 계셔서학습 스케줄 관...,요즘은 초등홈스쿨링을책이나 문제집으로 하는 지면학습보다스마트학습을 선호하는 분들이 ...,False,False,False


In [4]:
df.rename(columns={'text' : 'contents'}, inplace=True)
# 크롤링 실패한 raw, 불필요한 column 삭제
col_list = list(df.columns)
col_list.remove('contents')
df.drop(col_list, axis=1, inplace=True)
df

,contents
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...
...,...
1006,중학 스마트학습 천재교육 밀크티로. 온라인 학습하다가학교 다니고 하다 보니아이들이 ...
1007,홈쇼핑에서 2월 말부터 방송이 나오더라고요와이즈캠프도 있었는데 그건 신청을했어요우선...
1008,작년 이맘때 초등준비를 위해7세 홈스쿨로 무엇이 좋을까 고민 중이었어요. 학습적인 ...
1009,요즘은 초등홈스쿨링을책이나 문제집으로 하는 지면학습보다스마트학습을 선호하는 분들이 ...


### 함수 개선안

1. 문장 분리 단계에서 정규표현식을 사용하여 더욱 정확하게 문장을 분리할 수 있다.

1. 추출하려는 키워드가 포함된 문장들을 추출하는 과정에서 리스트 컴프리헨션을 사용하면 반복문을 대체할 수 있습니다.

1. 추출한 문장들을 다시 단어 단위로 분리하여 동사와 명사를 추출하는 과정에서도 리스트 컴프리헨션을 사용하여 반복문을 대체할 수 있습니다.

1. 병렬처리를 사용하여 분산 처리할 수 있습니다. 이를 위해서는 multiprocessing 라이브러리를 사용하여 병렬처리를 구현할 수 있습니다.

In [5]:
# dataframe에 있는 contents(본문)에 키워드 존재 여부 칼럼만들기
def keyword_search(df, key1):

    key1_list = []
    key2_list = []

    for i in df['contents']:
        if '아이스크림' in i:
            key1_list.append(True)

        else:
            key1_list.append(False)

    df[key1] = key1_list
    return df

# 전처리 함수 1 : point_reduce
# 점이 여러개 찍혀있는 것을 한개로 바꿈

def point_reduce(sentence):
    while True:
        try:
            if '..' in sentence:
                sentence = sentence.replace('..','.')
            
            else:
                sentence = sentence.replace('.','. ') # 온점 뒤에 space 넣어뒀음. 불필요시 제거.
                if '  ' in sentence:
                    sentence = sentence.replace('  ',' ')
                    # print(sentence)
                    return sentence
                    break
                else:
                    return sentence
        except:
            return sentence
        
        # if '  ' not in sentence:
        #     break

# 문장 단위로 끊기
def classify_sentence(sentence):
    sentence_list = sentence.split('. ')
    if ' ' in sentence_list:
        sentence_list.remove(' ')
        return sentence_list

    else:
        return sentence_list

# dummy 없애기 (밀크티 관련만 남기기)

def search_milkT(text):
    if "밀크티" in text:
        return True

    elif "밀크t" in text.lower():
        return True
            
    elif "milkT" in text.lower():
        return True
    
    elif "ㅁㅋㅌ" in text:
        return True

    # elif "학습" in text:
    #     return True

    else:
        return False

# 광고 여부 확인하기

def check_ad(text):
    if '광고' in text:
        return True
    
    elif '추천인' in text:
        return True

    elif '서포터' in text:
        return True

    else:
        return False

# 주어 찾기 함수

def subject_extraction(text):
    
    # 형태소 분석
    pos = Kkma.pos(text)

    # 주어 추출
    for i in range(len(pos)):
        if pos[i][1] == 'NNG':  # 명사
            if i == 0 or (i > 0 and pos[i-1][1] != 'NNG'):  # 주어는 문장 첫 단어이거나 앞 단어가 조사가 아닌 경우
                return pos[i][0]
                break



# 키워드가 주어인 본문 정리하기
## keword에서 다음 keyword 까지의 text 추출


def gather_sentence(text, key1, key2):
    
    text_list = text.split('.') # 문장 단위 : "."

    while ' ' in text_list: # '' 안에 text가 들어있지 않으면 error가 발생함을 방지
        text_list.remove(' ')

    key1_paragraph = []
    key2_paragraph = []
    for i in range(len(text_list)):
        if subject_extraction(text_list[i]) == key1:
            plus_num = 1
            # milk_para.append(text_list[i])
            sentence = text_list[i]
            # print(f'{key1}파트 {i}번째 진행중')
            while i+plus_num < len(text_list):
                if subject_extraction(text_list[i+plus_num]) == key1 or subject_extraction(text_list[i+plus_num]) == key2:
                    # print(f'!! 스캔 {key1}파트 종료')
                    break
                
                else:
                    # milk_para.append(text_list[i+plus_num])
                    if i+plus_num > len(text_list):
                        # print(f'!! 스캔 {key1}파트 종료')
                        break
                    else:
                        sentence = sentence + '' + text_list[i+plus_num]
                        plus_num += 1
                        # print(f'!! 스캔 {key1}파트 {i+plus_num}번째 진행중')

            key1_paragraph.append(sentence)

        else:
            pass

        if subject_extraction(text_list[i]) == key2:
            plus_num = 1
            # milk_para.append(text_list[i])
            sentence = text_list[i]
            # print(f'{key2}파트 {i}번째 진행중')
            while i+plus_num < len(text_list):
                if subject_extraction(text_list[i+plus_num]) == key1 or subject_extraction(text_list[i+plus_num]) == key2:
                    break
                
                else:
                    # milk_para.append(text_list[i+plus_num])
                    sentence = sentence + '' + text_list[i+plus_num]
                    plus_num += 1

            key2_paragraph.append(sentence)

        else:
            pass

    if key1_paragraph == []:
        for i in range(len(text_list)):
            pos_list = Kkma.pos(text_list[i])
            if (key1, 'NNG') in pos_list:
                key1_paragraph.append(text_list[i])

    if key2_paragraph == []:
        for i in range(len(text_list)):
            pos_list = Kkma.pos(text_list[i])
            if (key2, 'NNG') in pos_list:
                key2_paragraph.append(text_list[i])
    else:
        pass
    
    return {key1 : key1_paragraph, key2 : key2_paragraph}

# 명사, 동사 추출

def extract_func(dict):
    key1 = list(dict.keys())[0]
    key2 = list(dict.keys())[1]
    key1_list = []
    key2_list = []

    for i in range(len(dict[key1])):
        pos = Kkma.pos(dict[key1][i])
        for j in pos:
            if j[1] == 'VA': #동사
                if j[1] not in stop_words:
                    key1_list.append(j[0]+'다')
            if j[1] == 'NNG':
                if j[1] not in stop_words:
                    key1_list.append(j[0])

    for i in range(len(dict[key2])):
        pos = Kkma.pos(dict[key2][i])
        for j in pos:
            if j[1] == 'VA': #동사
                if j[1] not in stop_words:
                    key2_list.append(j[0]+'다')
            if j[1] == 'NNG':
                if j[1] not in stop_words:
                    key2_list.append(j[0])

    frequncy_dict = {key1 : key1_list, key2 : key2_list}

    return frequncy_dict

In [6]:
df['Check_milkT'] = df['contents'].apply(lambda x : search_milkT(x))
df['check_test'] = df['contents'].apply(lambda x : '학습' in x and '선생' in x and '모금' not in x)
# df.Check_milkT.value_counts()
# df['check_test'].value_counts()

df = df[df['check_test'] == True]
df = df[df['Check_milkT'] == True]

# df['check_test2'] = df['contents'].apply(lambda x : '선생' in x)
# df= df[df['check_test2'] == True]

df['recommend'] = df['contents'].apply(lambda x : check_ad(x))
df = df.drop(['Check_milkT', 'check_test'], axis=1)
df.reset_index(drop=True, inplace=True)

In [7]:
# # dataframe 보기 설정
# pd.set_option('display.max_row', 20)

In [8]:
df.contents = df.contents.apply(lambda x : re.sub('[-=+#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n★▲;]','', x))
df.contents = df.contents.apply(lambda x : re.sub('\W. ','',x))
df.contents = df.contents.apply(lambda x : core.replace_emoji(x, replace=''))
display(df[:10])
print(len(df))

,contents,recommend
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True
5,안녕하세요 태블리입니다 D오늘은 우리딸램이 하고 있는 아이스크림 홈런을 추천해드리려...,False
6,아이스크림 홈런 1년 반개월 정도 하고 끝냈다. 1학년 입학하고 나서 부터 했던지라...,True
7,초등수학 공부법 초2수학 교과연계 밀크T AI학습으로글. 사진 모나안녕하세요 모나에...,False
8,원래도4학년부터 과학이랑 사회가 어려워진대서학습지를 고려하던 차에말레이시아에서달 살...,True
9,지난 일이주일간 시중에 나와있는 여러 초등학생 학습용 스마트패드를 체험해보았습니다회...,False


1009


In [9]:
contents_list = list(df.contents)
s = contents_list[0]
s

'아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 밀크T가격 홈런가격 밀크T비용 홈런비용 밀크티기계 홈런기계 밀크티삼성 밀크T기계 밀크T삼성안녕하세요. 미즈츄입니다. 저희 아이 둘은 2년동안 윙크를 꾸준하게 해왔는데요. 큰아이 7살 작은아이 6살되는2월부터 가입해서 매일 빠짐 없이 해왔어요. 2년동안 하다보니 가끔 하기 싫어서 짜증내던 시기가 있었는데때는 계약기간까지만 하고 그만둬야겠다 생각했었는데그 시기가 지나니하더라고요. 아이들마다 성향이 다르니 아이의 성향을파악하시고 학습기를 통한 학습을지 구몬과 같이 선생님이 방문하거나 지면학습을지 선택하면것 같아요. 이제 둘째도 내년이면 학교에 들어가게되니 초등학생이만한 학습기를 찾던밀크T와 홈런을 알게되었습니다. 2가지가 가장 보편적으로 많이 하는 학습기더라고요.10무료체험 중이라 부담없이 비교해볼있었어요. 참고로 주말이 끼다보니 체감상 일주일 정도 학습하는 느낌이었어요. 얼마전 SK텔레콤 행사로 밀크T 무료체험2만원 상품권을 준다고 문자가 왔었는데 때마침 고민하고 있던 터라 바로 신청을 했어요. 신청그바로 담당선생님에게 전화가 왔고 이틀 정도 지나 기기를 받았습니다. 기기 외에도 이렇게 책도 넣어주셨네요. 밀크T는 로그인이 홈런보다는복잡한 편인데요. 로그인학습기를 켜면 밀크티하고소리가 나오는데 저는 기기를때마다소리가 거슬렸다는. ㅎㅎ바탕화면이 저렇게 나오는데 저는 초등으로 들어갔어요. 콘텐츠는 굉장히 많았는데요. 사고력수학이라는 콘텐츠가 있어서 조금심화해서 학교수업을 들여다수 있어 좋더라고요. 외에도 정말 콘텐츠가 많습니다. 솔직히 너무 많아서 저는제가 봤던 콘텐츠가 어디있었나 다시 찾기가 힘들었어요. ㅎㅎ밀크T를 일주일 체험했을 때쯤 아이스크림 홈런을 신청해 받아봤어요. 포장박스나 기기는 홈런이 훨씬 튼튼하고 예쁩니다. 따로 서비스로 주는같은없지만기계에 손잡이가 있어 이동편리하고 튼튼해서 첫인상은 홈런이좋았어요. 밀크T보다 로그인은 홈런이 훨씬 간단하고 편리합니다. 콘텐츠는 밀크

In [10]:
df2 = df.copy()
df2.head()

,contents,recommend
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True


In [11]:
df['contents'] = df['contents'].apply(lambda x: point_reduce(x))
df['contents_list'] = df['contents'].apply(lambda x: classify_sentence(x))
display(df['contents'][50])
display(df['contents_list'][50])

'1초딩 딸공이 밀크티만남이사마구잡이로 쌓아 정리된 책들이사아직 끝나지 않은정리에 연일 정신이 없는 나날이었다. 1초등 우리 딸공이의 학습과 독서 루틴이 무너진말할 것도 없다. 집에 새로 들일 책장이 아직 배송이된 터라서 책들이 마구잡이로 섞여 쌓여 있으니 딸아이가 평소 공부하던 문제집도 행방이 묘연한 상태. 그나저나 문제집을 찾아야 공부를터인데 중요하게 찾아야도장과 서류도찾고 있는 마당에 문제집 찾기는 자꾸 뒤로 밀려났다점이라고 손가락으로 가리킴 ㅋㅋ엄마 이리 와서 이것보세요100점이에요. 맞았어요그러던 차에 지난달에 신청해서 받아 두었던 밀크티 학습기기가 천재교육에서 만든 스마트 학습기기이다 떠올랐다. 이것이이제서야 생각났던 건지. 굳이 문제집이 없더라도학습기기로 편하게 공부할있는 것을 두고 그간 아이의 학습에 공백이 뚫리게 해버렸다. 언제 어디서든 기기만 있고 와이파이가 되는 곳이라면 학습이 가능하니 이런온라인 스마트 학습의 장점이기도 하다딸공이 하고 있는 태블릿과 비슷한있는데 이걸로 학교 공부도 하고 영어 공부도수 있대. 한번 해볼래이 말만을 하고 그냥 딸아이에게 건네주었다. 나도 아직 사용법을 숙지하지 않았고 자세한 콘텐츠까지는모르는 상태라 아이에게 자세히 알려주지도 못하는 상황이었으니까. 밀크티 학습 콘텐츠 구경 중딸좋아요 진짜거 태블릿이랑 비슷하네요딸은 공부를 하게 되어 기쁜 것은 전혀 아닐 테고 새로운 기기를 받아들고 마냥 좋은 모양이었다. 그러고는 처음엔 그냥 이것저것 살펴보더니 이내 수학 문제를 풀고는 100점 맞았다며 좋아한다. 처음에는 그냥 기기만 건네주고 혼자서 이것저것 살펴보라고 두었는데 어린아이들은 설명하지 않아도 이런 기기에 능숙하더라. 100점 칭찬도 하면서 내친김에 아이 곁에 앉아서 함께 밀크학습 콘텐츠를 살펴보기로 하였다. 많은 콘텐츠가 있지만가운데 우리는 간략히가지만 살펴보았다. 초등 1학년 홈스쿨 천재교육 밀크티 초등인강 맛보기평소 영어에 관심이 많은 나와우리 딸공이가 가장 먼저 살펴 본콘텐츠는 말할 필요 없이 영어그중에서

['1초딩 딸공이 밀크티만남이사마구잡이로 쌓아 정리된 책들이사아직 끝나지 않은정리에 연일 정신이 없는 나날이었다',
 '1초등 우리 딸공이의 학습과 독서 루틴이 무너진말할 것도 없다',
 '집에 새로 들일 책장이 아직 배송이된 터라서 책들이 마구잡이로 섞여 쌓여 있으니 딸아이가 평소 공부하던 문제집도 행방이 묘연한 상태',
 '그나저나 문제집을 찾아야 공부를터인데 중요하게 찾아야도장과 서류도찾고 있는 마당에 문제집 찾기는 자꾸 뒤로 밀려났다점이라고 손가락으로 가리킴 ㅋㅋ엄마 이리 와서 이것보세요100점이에요',
 '맞았어요그러던 차에 지난달에 신청해서 받아 두었던 밀크티 학습기기가 천재교육에서 만든 스마트 학습기기이다 떠올랐다',
 '이것이이제서야 생각났던 건지',
 '굳이 문제집이 없더라도학습기기로 편하게 공부할있는 것을 두고 그간 아이의 학습에 공백이 뚫리게 해버렸다',
 '언제 어디서든 기기만 있고 와이파이가 되는 곳이라면 학습이 가능하니 이런온라인 스마트 학습의 장점이기도 하다딸공이 하고 있는 태블릿과 비슷한있는데 이걸로 학교 공부도 하고 영어 공부도수 있대',
 '한번 해볼래이 말만을 하고 그냥 딸아이에게 건네주었다',
 '나도 아직 사용법을 숙지하지 않았고 자세한 콘텐츠까지는모르는 상태라 아이에게 자세히 알려주지도 못하는 상황이었으니까',
 '밀크티 학습 콘텐츠 구경 중딸좋아요 진짜거 태블릿이랑 비슷하네요딸은 공부를 하게 되어 기쁜 것은 전혀 아닐 테고 새로운 기기를 받아들고 마냥 좋은 모양이었다',
 '그러고는 처음엔 그냥 이것저것 살펴보더니 이내 수학 문제를 풀고는 100점 맞았다며 좋아한다',
 '처음에는 그냥 기기만 건네주고 혼자서 이것저것 살펴보라고 두었는데 어린아이들은 설명하지 않아도 이런 기기에 능숙하더라',
 '100점 칭찬도 하면서 내친김에 아이 곁에 앉아서 함께 밀크학습 콘텐츠를 살펴보기로 하였다',
 '많은 콘텐츠가 있지만가운데 우리는 간략히가지만 살펴보았다',
 '초등 1학년 홈스쿨 천재교육 밀크티 초등인강 맛보기평소 영어에 관

In [12]:
# vector = CountVectorizer()
# bow = vector.fit_transform()

# print(bow.toarray())
# print(vector.vocabulary_)

In [13]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
# !pip install konlpy

### mecab install try
## ! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
## ! bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh
## !pip install eunjeon

In [14]:
from konlpy.tag import Kkma, Hannanum, Komoran, Twitter, Okt

In [15]:
Kkma = Kkma()
okt = Okt()
Komoran = Komoran()
hannanum = Hannanum()
twitter = Twitter()
# mecab = Mecab()

c:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
df.head()

,contents,recommend,contents_list
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False,[아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기...
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True,[우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이...
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False,[곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주...
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False,[안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습...
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True,[앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다...


In [17]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(s)
tokens

['아이스크림홈런',
 '천재교육밀크티',
 '밀크T',
 '밀크티',
 '초등학습지',
 '유아학습지',
 '학습지선택',
 '학습기',
 '밀크T가격',
 '홈런가격',
 '밀크T비용',
 '홈런비용',
 '밀크티기계',
 '홈런기계',
 '밀크티삼성',
 '밀크T기계',
 '밀크T삼성안녕하세요',
 '.',
 '미즈츄입니다',
 '.',
 '저희',
 '아이',
 '둘은',
 '2년동안',
 '윙크를',
 '꾸준하게',
 '해왔는데요',
 '.',
 '큰아이',
 '7살',
 '작은아이',
 '6살되는2월부터',
 '가입해서',
 '매일',
 '빠짐',
 '없이',
 '해왔어요',
 '.',
 '2년동안',
 '하다보니',
 '가끔',
 '하기',
 '싫어서',
 '짜증내던',
 '시기가',
 '있었는데때는',
 '계약기간까지만',
 '하고',
 '그만둬야겠다',
 '생각했었는데그',
 '시기가',
 '지나니하더라고요',
 '.',
 '아이들마다',
 '성향이',
 '다르니',
 '아이의',
 '성향을파악하시고',
 '학습기를',
 '통한',
 '학습을지',
 '구몬과',
 '같이',
 '선생님이',
 '방문하거나',
 '지면학습을지',
 '선택하면것',
 '같아요',
 '.',
 '이제',
 '둘째도',
 '내년이면',
 '학교에',
 '들어가게되니',
 '초등학생이만한',
 '학습기를',
 '찾던밀크T와',
 '홈런을',
 '알게되었습니다',
 '.',
 '2가지가',
 '가장',
 '보편적으로',
 '많이',
 '하는',
 '학습기더라고요.10무료체험',
 '중이라',
 '부담없이',
 '비교해볼있었어요',
 '.',
 '참고로',
 '주말이',
 '끼다보니',
 '체감상',
 '일주일',
 '정도',
 '학습하는',
 '느낌이었어요',
 '.',
 '얼마전',
 'SK텔레콤',
 '행사로',
 '밀크T',
 '무료체험2만원',
 '상품권을',
 '준다고',
 '문자가',
 '왔었는데',
 '때마침',
 '고민하고',
 '있던',
 '터라',
 '바

In [18]:
print('Kkma 형태소 분석 : ', Kkma.pos(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.pos(s[:20]))
print('Komoran 형태소 분석 : ', Komoran.pos(s[:20]))
print('Twitter 형태소 분석 : ', twitter.pos(s[:20]))
print('Okt 형태소 분석 : ', okt.pos(s[:20]))
# print('Mecab 형태소 분석 : ', mecab.pos(sentence))

# '3학년', '밀크T'를 묶는 모습에서 Hannaum이 일단은 제일 나아보인다

Kkma 형태소 분석 :  [('아이스크림', 'NNG'), ('홈런', 'NNG'), ('천재', 'NNG'), ('교육', 'NNG'), ('밀크', 'NNG'), ('티', 'NNG'), ('밀크', 'NNG'), ('T', 'OL')]
Hannaum 형태소 분석 :  [('아이스크림홈런', 'N'), ('천재교육밀크티', 'N'), ('밀크T', 'N')]
Komoran 형태소 분석 :  [('아이스크림', 'NNP'), ('홈런', 'NNP'), ('천재교육', 'NNP'), ('밀크티', 'NNP'), ('밀크', 'NNP'), ('T', 'SL')]
Twitter 형태소 분석 :  [('아이스크림', 'Noun'), ('홈런', 'Noun'), ('천재교육', 'Noun'), ('밀크', 'Noun'), ('티', 'Noun'), ('밀크', 'Noun'), ('T', 'Alpha')]
Okt 형태소 분석 :  [('아이스크림', 'Noun'), ('홈런', 'Noun'), ('천재교육', 'Noun'), ('밀크', 'Noun'), ('티', 'Noun'), ('밀크', 'Noun'), ('T', 'Alpha')]


In [19]:
# 키워드 찾기 기능?!
# 밀크T 가 온전히 나오는게 나쁘지 않을듯 하기도 하구...
# 혹은 n-gram추출을 통해서 밀크+T 아이스크림+홈런 같은 식으로 검색을 진행해봐도 괜찮을 듯 하다.

print('Kkma 형태소 분석 : ', Kkma.nouns(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.nouns(s[:20]))
print('Komoran 형태소 분석 : ', Komoran.nouns(s[:20]))
print('Twitter 형태소 분석 : ', twitter.nouns(s[:20]))
print('Okt 형태소 분석 : ', okt.nouns(s[:20]))
# print('Mecab 형태소 분석 : ', mecab.nouns(sentence))
print('\n--------------------------------------------\n')

ex_sentence = '1학년부터 시작해서 6학년 그리고 중학교 전학년까지 가능합니다.'
# ex_sentence = '밀크T의 특징과 아이스크림 홈런의 특징 그리고 엘리하이의 특징에 대해서 알아보겠습니다.'
print('Kkma 형태소 분석 : ', Kkma.nouns(ex_sentence))
print('Hannaum 형태소 분석 : ', hannanum.nouns(ex_sentence))
print('Komoran 형태소 분석 : ', Komoran.nouns(ex_sentence))
print('Twitter 형태소 분석 : ', twitter.nouns(ex_sentence))
print('Okt 형태소 분석 : ', okt.nouns(ex_sentence))

print('\n-------------------------------------------\n')

bigram = list(ngrams(ex_sentence.split(), 2))
print(bigram)
blob = TextBlob(ex_sentence)
blob.ngrams(n=2)

Kkma 형태소 분석 :  ['아이스', '아이스크림', '아이스크림홈런', '크림', '홈런', '천재', '천재교육밀크티', '교육', '밀크', '티']
Hannaum 형태소 분석 :  ['아이스크림홈런', '천재교육밀크티', '밀크T']
Komoran 형태소 분석 :  ['아이스크림', '홈런', '천재교육', '밀크티', '밀크']
Twitter 형태소 분석 :  ['아이스크림', '홈런', '천재교육', '밀크', '티', '밀크']
Okt 형태소 분석 :  ['아이스크림', '홈런', '천재교육', '밀크', '티', '밀크']

--------------------------------------------

Kkma 형태소 분석 :  ['1', '1학년', '학년', '6', '6학년', '중학교', '가능']
Hannaum 형태소 분석 :  ['1학년', '시작해서', '6학년', '중학교', '전학년', '가능']
Komoran 형태소 분석 :  ['학년', '시작', '학년', '중학교', '전', '학년']
Twitter 형태소 분석 :  ['학년', '시작', '학년', '중학교', '학년']
Okt 형태소 분석 :  ['학년', '시작', '학년', '중학교', '학년']

-------------------------------------------

[('1학년부터', '시작해서'), ('시작해서', '6학년'), ('6학년', '그리고'), ('그리고', '중학교'), ('중학교', '전학년까지'), ('전학년까지', '가능합니다.')]


[WordList(['1학년부터', '시작해서']),
 WordList(['시작해서', '6학년']),
 WordList(['6학년', '그리고']),
 WordList(['그리고', '중학교']),
 WordList(['중학교', '전학년까지']),
 WordList(['전학년까지', '가능합니다'])]

### 단어 빈출 분석

#### 방법
1. 명사만 추출하여 빈출어 정리
    - 불용어 리스트를 만든 뒤, 불용어를 제외하여 형태소 분석 진행
    - 단어 빈도 추출
    - 단어들만 : 밀크T, 아이스크림, 태블릿, 
    - 단어 빈도 시각화
        - barchar
        - treemap
        - wordcloud
        <br>
1. 형태소 분석을 통해 각 문장에서 명사 / 형용사 등을 추출하여서 긍정적/부정적 키워드 분류 및 통계분석 (ex. 밀크T : 재밌게 하네요)

### 우선 Hannanum 으로 진행

### Hannanum
- 태그
    - N : 명사
        - NC : 보통명사
        - NQ : 고유명사
        - NB : 의존명사
        - NP : 대명사
        - NN : 수사
    
    - P : 용언
        - PV : 동사
        - PA : 형용사
        - PX : 보조용언

    - M : 수식언
        - MM : 관형사
        - MA : 부사
    
    - I : 독립언
    - J : 관계언
        - JC : 격조사
        - JX : 보조사
        - JP : 서술격조사

    - E : 어미
    - X : 접사

In [20]:
print('Hannaum 형태소 분석 : ', hannanum.nouns(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.pos(s[:20]))
a = hannanum.nouns(s[:20])
b = hannanum.pos(s[:20])

Hannaum 형태소 분석 :  ['아이스크림홈런', '천재교육밀크티', '밀크T']
Hannaum 형태소 분석 :  [('아이스크림홈런', 'N'), ('천재교육밀크티', 'N'), ('밀크T', 'N')]


In [21]:
print(b)
print(b[0][0], b[0][1])

[('아이스크림홈런', 'N'), ('천재교육밀크티', 'N'), ('밀크T', 'N')]
아이스크림홈런 N


In [22]:
ex_sentence2 = '밀크T의 장점은 편하고 아이스크림 홈런의 장점은 귀엽고 엘리하이의 단점은 어려운 것 같아요'

In [23]:
stop_words = ['아이', '맘', '거', '저', '저희', '것', '학모', '감사', '게', '때', '애', '원', '제', '개', '일', '듯', '만', '번', '데', '분', '건', '정도', '달', '정보', '이번', '부탁', '걸로', '후', '나', '부분', '걸', '저흰', '답변', '마음', '뼈', '건가요', '오늘', '왜', '시', '꺼', '친구', '마미', '때문', '하루', '명', '돈', '해요', '이', '가요', '기', '지금', '건가요', '반', '주세요', '편', '외', '소개', '입니다', '곳', '이거', '뭐', '땐', '건지', '그거', '점', '땐', '미', '풀', '대', '안', '눈', '공주', '사', '오', '백', '후', '나']

In [24]:
test_noun = []

for noun in tqdm(Kkma.nouns(df.contents[0])):
    if noun not in stop_words:
        test_noun.append(noun)

test_count = Counter(test_noun)
test_top_nouns = dict(test_count.most_common(100))
test_top_nouns

100%|██████████| 316/316 [00:00<?, ?it/s]


{'내': 2,
 '아이스': 1,
 '아이스크림': 1,
 '아이스크림홈런': 1,
 '크림': 1,
 '홈런': 1,
 '천재': 1,
 '천재교육밀크티': 1,
 '교육': 1,
 '밀크': 1,
 '티': 1,
 '밀크티': 1,
 '초등': 1,
 '초등학습지': 1,
 '학습지': 1,
 '유아': 1,
 '유아학습지': 1,
 '학습지선택': 1,
 '선택': 1,
 '학습': 1,
 '학습기': 1,
 '가격': 1,
 '홈런가격': 1,
 '비용': 1,
 '홈런비용': 1,
 '밀크티기계': 1,
 '기계': 1,
 '홈런기계': 1,
 '밀크티삼성': 1,
 '삼성': 1,
 '삼성안녕': 1,
 '안녕': 1,
 '미즈': 1,
 '미즈츄': 1,
 '츄': 1,
 '둘': 1,
 '2': 1,
 '2년동안': 1,
 '년': 1,
 '동안': 1,
 '윙크': 1,
 '큰아이': 1,
 '7': 1,
 '7살': 1,
 '살': 1,
 '6': 1,
 '6살': 1,
 '2월': 1,
 '월': 1,
 '가입': 1,
 '하다': 1,
 '짜증': 1,
 '짜증내': 1,
 '시기': 1,
 '계약': 1,
 '계약기간': 1,
 '기간': 1,
 '겠다': 1,
 '생각': 1,
 '성': 1,
 '성향이': 1,
 '향이': 1,
 '성향': 1,
 '파악': 1,
 '통': 1,
 '과': 1,
 '선생님': 1,
 '방문': 1,
 '선택하면것': 1,
 '이제': 1,
 '둘째': 1,
 '내년': 1,
 '학교': 1,
 '초등학생': 1,
 '2가지': 1,
 '가지': 1,
 '보편적': 1,
 '고': 1,
 '10': 1,
 '10무료체험': 1,
 '무료': 1,
 '체험': 1,
 '중': 1,
 '부담': 1,
 '비교': 1,
 '볼': 1,
 '참고': 1,
 '주말': 1,
 '체감': 1,
 '일주일': 1,
 '느낌': 1,
 '얼마': 1,
 '얼마전': 1,
 '전': 1,
 '텔레콤': 1,
 '행사

In [25]:
test_dict = {}
test_dict['a'] = ['a']
print(test_dict.get('a'))

test_dict.get('a').append('b')
print(test_dict.get('a'))

['a']
['a', 'b']


In [26]:
icecream_list = []
elihigh_list = []

for i in df['contents']:
    if '아이스크림' in i:
        icecream_list.append(True)

    else:
        icecream_list.append(False)

for i in df['contents']:
    if '엘리하' in i:
        elihigh_list.append(True)

    else:
        elihigh_list.append(False)

In [27]:
df_2 = df.copy()
df_2.drop('contents_list', axis=1, inplace=True)
df_2
# keyword_search(df, '아이스크림')

,contents,recommend
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True
...,...,...
1004,중학 스마트학습 천재교육 밀크티로. 온라인 학습하다가학교 다니고 하다 보니아이들이 ...,False
1005,홈쇼핑에서 2월 말부터 방송이 나오더라고요와이즈캠프도 있었는데 그건 신청을했어요우선...,False
1006,작년 이맘때 초등준비를 위해7세 홈스쿨로 무엇이 좋을까 고민 중이었어요. 학습적인 ...,False
1007,요즘은 초등홈스쿨링을책이나 문제집으로 하는 지면학습보다스마트학습을 선호하는 분들이 ...,False


In [28]:
df.drop('contents_list', axis=1, inplace=True)
df = keyword_search(df, '아이스크림')
df = keyword_search(df, '엘리하이')   

In [29]:
df_test = df[df['엘리하이'] == True]
df_test = df_test[df_test['아이스크림'] == True]
df_test.reset_index(drop=True, inplace=True)
df_test

,contents,recommend,아이스크림,엘리하이
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False,True,True
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True,True,True
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False,True,True
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False,True,True
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True,True,True
...,...,...,...,...
203,새학기를 앞두고 오늘은 스마트 학습기 비교를 위해 각각의 특징과 관리 형태 등을 안...,False,True,True
204,필요했던 시간이 있었다. 개척교회를 섬기게 되면서아이들끼리 있어야할 시간. 개인적으...,False,True,True
205,밀크티 홈런 비교 후기밀크티아이스크림홈런밀크티홈런‮ 아마 예비초딩 혹은 초2 단계에...,False,True,True
206,초등학생 인강홈런 VS 밀크티 온라인학습 비교 무료체험 솔직 후기 홈런 오른 밀크티...,True,True,True


In [30]:
sentence_96 = df_test['contents'][3]
sentence_96[1000:1500]

'이네요밀크t의 경우 오늘의 학습에 배정받으신 담임선생님께서 교과서 연계 학습을 넣어주시는데요교과목외에도 독서논술을 추가로 넣어주셔서 논술 학습을 함께 진행할있도록 도움을 줍니다타사와 다르게 밀크t의 경우 3학년부터 중국어를 학습할있도록 선택이 가능해요물론 2학년이던 1학년이던 열려있기에 배우고 싶은 시기에 3학년 과정으로 넘어가서 바로 학습은 가능합니다중국어도 시작 해야하나 싶은 생각이 있었는데 중국어도 담겨있어서 추가요금없이 함께 진행할있다는 점이 마음에 들었어요3학년부터 배우는 영어의 경우 검정교과서가 아니라 학교마다 차이가 있는데요어린이 학교는 천재교육 교과서를 사용하기에 영어수업을 집에서 예복습하기에맞는 학습기기였어요교과서도 만들고 해법수학 해법영어 셀파우등생 등을 진행하는 천재교육에서 만들어진 스마트 학습기기라 다양한 컨텐츠부터 문제출제방식 모두 마음에 들었어요첨삭과외라는 컨텐츠의 경우 서술형의 답을 하면 서술담당 선생님께서 확인체점까지 진행해주십니다요즘 수학도 단답형만이 '

### 설계해보자...

key1 = '밀크'
key2 = '홈런'

1. key값이 나올때마다 문장을 끊어준다
    - key값이 목적어처럼 쓰이는 경우는...? -> 형태소분석 파트에서 주어를 분석해주는 기능을 사용해야 할 듯 하다.
    - solution1
        - key1 값이 나올때마다 split을 사용하여 문장을 끊어준다.
        - if문을 사용하여 split된 문장 중에 key2 값이 있다면, 따로 리스트에 추가한다.
    
2. 나눠진 문장을 리스트에 넣어준다 (split 사용시 skip)
3. 해당 리스트를 반복문을 사용하여 형태소 분석을 실시한다.
    - kkma를 사용하여 동사/형용사를 추출한다.
        - 속도가 느리다면 hannanum으로 실행하고 P 항목에만 kkma를 사용한다.
1. key값 별로 출현한 동사/형용사를 구분지어준다.
1. 단어별 빈도를 추출한다.

In [31]:
subject_extraction('밀크티 태블릿이 홈런 태블릿보다 좋다')

'밀크'

In [32]:
text_list = ['밀크티마이쪄', '응애', '밀크티배고파', '밀크태블릿', '밀농사', '어흥', '밀크티 태블릿이 홈런 태블릿보다 좋다', '우와', '홈런을 칩니다', '최고야', '밀크티학습']
text_test = ''

for i in text_list:
    text_test = text_test + '. ' + i

text_test

'. 밀크티마이쪄. 응애. 밀크티배고파. 밀크태블릿. 밀농사. 어흥. 밀크티 태블릿이 홈런 태블릿보다 좋다. 우와. 홈런을 칩니다. 최고야. 밀크티학습'

In [33]:
gather_sentence(text_test, '밀크', '홈런')

{'밀크': [' 밀크티마이쪄 응애',
  ' 밀크티배고파',
  ' 밀크태블릿 밀농사 어흥',
  ' 밀크티 태블릿이 홈런 태블릿보다 좋다 우와',
  ' 밀크티학습'],
 '홈런': [' 홈런을 칩니다 최고야']}

In [34]:
coffee = '밀크T 수업을 시작하고 1년이 지났다. 연장은 2년을 하였으며 처음엔 수학과목만 생각해서 시작을 했는데 홈런보다 영어 컨텐츠가 더 좋은 것 같아서 연장했다. 학교가기 전에 밀크티로 예습하면 수업이 재미있다고 한다. 아이들 말로는 홈런은 캐릭터가 재미있다고 한다. 홈런이 캐릭터 하나는 기가막히게 만드나보다. 혹시라도 홈런과 밀크티를 고민하는 분이 있으면 이 글을 참고해보세요.'
gather_sentence(coffee, '밀크', '홈런')

{'밀크': ['밀크T 수업을 시작하고 1년이 지났다 연장은 2년을 하였으며 처음엔 수학과목만 생각해서 시작을 했는데 홈런보다 영어 컨텐츠가 더 좋은 것 같아서 연장했다 학교가기 전에 밀크티로 예습하면 수업이 재미있다고 한다 아이들 말로는 홈런은 캐릭터가 재미있다고 한다'],
 '홈런': [' 홈런이 캐릭터 하나는 기가막히게 만드나보다', ' 혹시라도 홈런과 밀크티를 고민하는 분이 있으면 이 글을 참고해보세요']}

In [35]:
test_def = df.contents[0]
gather_sentence(test_def, '밀크', '홈런')

{'밀크': [' 밀크T는 로그인이 홈런보다는복잡한 편인데요 로그인학습기를 켜면 밀크티하고소리가 나오는데 저는 기기를때마다소리가 거슬렸다는 ㅎㅎ바탕화면이 저렇게 나오는데 저는 초등으로 들어갔어요 콘텐츠는 굉장히 많았는데요 사고력수학이라는 콘텐츠가 있어서 조금심화해서 학교수업을 들여다수 있어 좋더라고요 외에도 정말 콘텐츠가 많습니다 솔직히 너무 많아서 저는제가 봤던 콘텐츠가 어디있었나 다시 찾기가 힘들었어요',
  ' ㅎㅎ밀크T를 일주일 체험했을 때쯤 아이스크림 홈런을 신청해 받아봤어요 포장박스나 기기는 홈런이 훨씬 튼튼하고 예쁩니다 따로 서비스로 주는같은없지만기계에 손잡이가 있어 이동편리하고 튼튼해서 첫인상은 홈런이좋았어요',
  ' 밀크T보다 로그인은 홈런이 훨씬 간단하고 편리합니다 콘텐츠는 밀크티보다 한눈에 보기 좋고입장에서더 정돈된 느낌이었어요 원하는 콘텐츠를 찾기 쉬웠고 밀크T보다는 콘텐츠가 조금 적지만부분은 문제가 되진 않는같아요',
  ' 밀크T는 삼성 기기를 사용하고 홈런은 홈런 자체 기기인같았는데 밀크T는 자꾸 키보드판이 덜렁대고 이동손잡이가 없어불편했고요',
  ' 밀크T는 펜이 너무 작은 감이 있어요 체험용말고 나중에 정회원이 되어서기기의 펜은 홈런펜만큼 크기도 컸고 색상은 블랙',
  ' 밀크T였습니다 저희아이는 홈런이좋다하고 둘째는 밀크T가좋다고했어요 선택한 이유는 사실 콘텐츠 자체는다 모두 훌륭하기 때문에부분으로 선택하지는 않았구요',
  ' 밀크T가 친구추천 이벤트를 하더라고요 ㅎㅎ추천받는 분과 추천하는 분에게 상품권 10만원씩 제공되고 비용도 지금 이벤트 중이라 밀크T가저렴했어요',
  ' 밀크T는 24개월 약정 109000원이고 홈런은 24개월 약정 114000원입니다 원래는 밀크T의 경우 24개월 약정월 119000원이고 12개월 약정월 149000원인데요 현재 12월 이벤트로 109000원으로 하고 있어서부분을 놓칠 수가 없었어요 콘텐츠에차이가 없다면 혜택이 많은 쪽을 선택할밖에 없는 소비자 너란 사람 하아 ㅋㅋ사실 기계는 홈런

In [36]:
split_milk = sentence_96.split('밀크')
list_milk = []
list_homerun = []

for i in split_milk:
    if '홈런' in i:
        list_homerun.append(i)
    else:
        list_milk.append(i)
        # list_milk.append(Kkma.pos(i))

In [37]:
# text = "밀크티 무료체험 해보니 쉽고 좋아요 이제 해지하려구요"
text = "밀크티 무료체험 해보니 아이가 게임을 하며 즐거워하는데 너무 게임만 할까봐 걱정이네요."

pos = Kkma.pos(text)
subject_extraction(text=text)

'밀크'

In [38]:
Kkma.tagset
# VA : 형용사
# VV : 동사 - 얜 추출 안해도 될듯...?
# NNG : 보통명사
# VA, NNG만 추출

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [39]:
hannanum.pos(text)

[('밀크티', 'N'),
 ('무료체험', 'N'),
 ('하', 'P'),
 ('어', 'E'),
 ('보', 'P'),
 ('니', 'E'),
 ('아이', 'N'),
 ('가', 'J'),
 ('게임', 'N'),
 ('을', 'J'),
 ('하', 'P'),
 ('며', 'E'),
 ('즐겁', 'P'),
 ('어', 'E'),
 ('하', 'P'),
 ('ㄴ데', 'E'),
 ('너무', 'M'),
 ('게임', 'N'),
 ('만', 'J'),
 ('할까봐', 'N'),
 ('걱정', 'N'),
 ('이', 'J'),
 ('네', 'E'),
 ('요', 'J'),
 ('.', 'S')]

In [40]:
Kkma.nouns(text)

['밀크', '밀크티', '티', '무료', '무료체험', '체험', '아이', '게임', '걱정']

In [41]:
test = ' 밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음 처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다 진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면 일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음 ㅎㅎ그래서 우리남매는 모르는 계속 몰랐으면 한다3년하고 1개월이 지난같다 여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다 전원을 켜면 배터리는 다시 0로 꺼짐 학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다 충전이 어려울같아서 학습도 안되고 수업도 안될같다 담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다 15883366배터리 교체 비용을 물으니 66,000원이라고 한다 더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다 배터리 재고는하나 있다고 하셨다 재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다 하나 남은 재고 예약까지 했다 2주 이내로 방문하면 된다고 함 예약하고 다음날 바로 감 배터리 교체하고 학습해야 하니까 오후에 가니 확실히붐볐다 대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다 배터리 교체 비용 66,000원 결제 완료 사실, 1년 남은 시점이고 내년엔 중등으로 넘어가야 한다 아직까진 중학생이 되어도 밀크T를 계속할 생각이다 그래서패드로 교체를생각도 했는데6학년이지금 학교 마치고 방과후수업과 영어, 수학 학원을 다녀오면 7시시간이 있을까라는 생각이 든다 아직 6학년의 생활을 시작도했고봐야것 같아서남은 1년 동안 계속 유지가 될지 중간에 그만둘지 아무도 모른다 그래서패드로 교환보다는 배터리 교체를 했다 5학년 초반까지만 해도 스스로 자기주도학습이되었는데 점점 하라고 하라고 해야 하는 중 그래서 얼마나할있을지 모르겠다 그렇지만, 학습하는 내용은 아주 도움이 되니까, 계속 유지하길 바라는마음 서비스센터가 집에서 가까워서 막내 유모차 태우고 걸어갔는데,왕복 30분 거리를 걸어갔다 왔더니캐시워크가 오늘 하루 만보를 걸었단다 괜히 뿌듯'

In [42]:
test_kk = Kkma.pos(test)

In [43]:
# dict_list = []

# for i in range(len(df.contents)):
#     print(f'{i}번째 실행중')
#     dict_list.append(gather_sentence(df.contents[i], '밀크', '홈런'))

In [44]:
df

,contents,recommend,아이스크림,엘리하이
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False,True,True
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True,True,True
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False,True,True
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False,True,True
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True,True,True
...,...,...,...,...
1004,중학 스마트학습 천재교육 밀크티로. 온라인 학습하다가학교 다니고 하다 보니아이들이 ...,False,False,False
1005,홈쇼핑에서 2월 말부터 방송이 나오더라고요와이즈캠프도 있었는데 그건 신청을했어요우선...,False,True,True
1006,작년 이맘때 초등준비를 위해7세 홈스쿨로 무엇이 좋을까 고민 중이었어요. 학습적인 ...,False,False,False
1007,요즘은 초등홈스쿨링을책이나 문제집으로 하는 지면학습보다스마트학습을 선호하는 분들이 ...,False,False,False


In [45]:
df2 = df[:100].copy()

In [46]:
df2['keyword_dict'] = [gather_sentence(x, '밀크', '천재') for x in tqdm(df2['contents'])]

 97%|█████████▋| 97/100 [26:00<02:01, 40.58s/it]

In [ ]:
# df['keyword_dict'] = [gather_sentence(x, '밀크', '천재') for x in tqdm(df['contents'])]
# # text 길이마다 걸리는 시간은 다를수 있음. (1~5번째 content 하는데 90초가 걸려버림,,,)

In [ ]:
stop_words = ['아이', '맘', '거', '저', '저희', '것', '학모', '감사', '게', '때', '애', '원', '제', '개', '일', '듯', '만', '번', '데', '분', '건', '정도', '달', '정보', '이번', '부탁', '걸로', '후', '나', '부분', '걸', '저흰', '답변', '마음', '뼈', '건가요', '오늘', '왜', '시', '꺼', '친구', '마미', '때문', '하루', '명', '돈', '해요', '이', '가요', '기', '지금', '건가요', '반', '주세요', '편', '외', '소개', '입니다', '곳', '이거', '뭐', '땐', '건지', '그거', '점', '땐', '미', '풀', '대', '안', '눈', '공주', '사', '오', '백', '후', '나']

In [ ]:
first10sentence = list(df.contents[:10])
result_list = []

for i in first10sentence:
    i = i.replace('1','일')
    i = i.replace('2','이')
    i = i.replace('3','삼')
    i = i.replace('4','사')
    i = i.replace('5','오')
    i = i.replace('6','육')
    result_list.append(i)

In [ ]:
hannanum.pos(df.contents[0])

[('아이스크림홈런', 'N'),
 ('천재교육밀크티', 'N'),
 ('밀크T', 'N'),
 ('밀크티', 'N'),
 ('초등학습지', 'N'),
 ('유아학습지', 'N'),
 ('학습지선택', 'N'),
 ('학습기', 'N'),
 ('밀크T가격', 'N'),
 ('홈런가격', 'N'),
 ('밀크T비용', 'N'),
 ('홈런비용', 'N'),
 ('밀크티기계', 'N'),
 ('홈런기계', 'N'),
 ('밀크티삼성', 'N'),
 ('밀크T기계', 'N'),
 ('밀크T삼성안녕하세요', 'N'),
 ('.', 'S'),
 ('미즈츄입니', 'N'),
 ('이', 'J'),
 ('다', 'E'),
 ('.', 'S'),
 ('저희', 'N'),
 ('아이', 'N'),
 ('둘', 'N'),
 ('은', 'J'),
 ('2년', 'N'),
 ('동안', 'X'),
 ('윙크', 'N'),
 ('를', 'J'),
 ('꾸준', 'N'),
 ('하', 'X'),
 ('게', 'E'),
 ('하', 'P'),
 ('어', 'E'),
 ('오', 'P'),
 ('아ㄴ데', 'E'),
 ('요', 'J'),
 ('.', 'S'),
 ('큰아이', 'N'),
 ('7살', 'N'),
 ('작은아', 'N'),
 ('이', 'J'),
 ('6살되는2월부터', 'N'),
 ('가입해', 'N'),
 ('서', 'J'),
 ('매일', 'M'),
 ('빠지', 'P'),
 ('ㅁ', 'E'),
 ('없이', 'M'),
 ('하', 'P'),
 ('어', 'E'),
 ('오', 'P'),
 ('아아', 'E'),
 ('요', 'J'),
 ('.', 'S'),
 ('2년', 'N'),
 ('동안', 'X'),
 ('하', 'P'),
 ('어다', 'E'),
 ('보', 'P'),
 ('니', 'E'),
 ('가끔', 'M'),
 ('하', 'P'),
 ('기', 'E'),
 ('싫', 'P'),
 ('어서', 'E'),
 ('짜증내', 'N'),
 ('이', 'J')

In [ ]:
hannanum.nouns(df.contents[0])

['아이스크림홈런',
 '천재교육밀크티',
 '밀크T',
 '밀크티',
 '초등학습지',
 '유아학습지',
 '학습지선택',
 '학습기',
 '밀크T가격',
 '홈런가격',
 '밀크T비용',
 '홈런비용',
 '밀크티기계',
 '홈런기계',
 '밀크티삼성',
 '밀크T기계',
 '밀크T삼성안녕하세요',
 '미즈츄입니',
 '저희',
 '아이',
 '둘',
 '2년',
 '윙크',
 '꾸준',
 '큰아이',
 '7살',
 '작은아',
 '6살되는2월부터',
 '가입해',
 '2년',
 '짜증내',
 '시',
 '데',
 '계약기간',
 '생각했었는데그',
 '시',
 '지나니하더라고요',
 '아이들',
 '성향',
 '아이',
 '성향을파악하시',
 '학습기',
 '학습을지',
 '구몬',
 '선생님',
 '방문',
 '지면학습을지',
 '선택하면것',
 '둘째',
 '내년',
 '학교',
 '초등학생이만',
 '학습기',
 '찾던밀크T',
 '홈런',
 '2가지',
 '보편적',
 '학습기',
 '10무료체험',
 '중',
 '비교해볼있었어요',
 '참고',
 '주말',
 '체감상',
 '일주일',
 '정도',
 '학습',
 '느낌',
 '얼마전',
 '텔레콤',
 '행사',
 '밀크T',
 '무료체험2만원',
 '상품권',
 '문자',
 '고민',
 '터',
 '신청',
 '신청그바로',
 '담당선생님',
 '전화',
 '이틀',
 '정도',
 '기',
 '기',
 '외',
 '책',
 '밀크T',
 '로그인',
 '홈런보다는복잡한',
 '편인데요',
 '로그인학습기',
 '밀크티하고소리',
 '저',
 '기기를때마다소리',
 'ㅎㅎ바탕화면',
 '저',
 '초등',
 '콘텐츠',
 '사고력수학',
 '콘텐츠',
 '조금심화해서',
 '학교수업',
 '들여다수',
 '외',
 '콘텐츠',
 '저는제',
 '콘텐츠',
 '어디있었나',
 'ㅎㅎ밀크T',
 '일주일',
 '체험',
 '때쯤',
 '아이스크림',
 '홈런',
 '신청',
 '포장박스',
 '홈런'

--임시 분석용(100개 데이터)--

In [ ]:
test_noun2 = []

for text in tqdm(df2.contents):
    # for noun in Kkma.nouns(text): #hannanum이 훨씬 빠르다 (Kkma : 3초에 1문장, hannanum : 1초에 5~10문장)
    for noun in Kkma.nouns(text):
        if noun not in stop_words:
            test_noun2.append(noun)

test_count2 = Counter(test_noun2)
test_top_nouns2 = dict(test_count2.most_common(100))
test_top_nouns2

100%|██████████| 100/100 [04:18<00:00,  2.59s/it]


{'수': 150,
 '밀크': 100,
 '학습': 100,
 '선생님': 100,
 '아이스': 93,
 '아이스크림': 93,
 '크림': 93,
 '체험': 86,
 '티': 85,
 '1': 85,
 '공부': 83,
 '선택': 81,
 '시간': 81,
 '2': 80,
 '중': 79,
 '천재': 78,
 '교육': 78,
 '우리': 78,
 '수학': 77,
 '등': 77,
 '밀크티': 74,
 '초등': 74,
 '영어': 73,
 '생각': 72,
 '무료': 70,
 '하나': 69,
 '3': 68,
 '학교': 67,
 '다양': 67,
 '교과서': 67,
 '가능': 66,
 '학년': 65,
 '스마트': 65,
 '문제': 64,
 '년': 62,
 '전': 61,
 '신청': 61,
 '홈런': 59,
 '무료체험': 59,
 '처음': 59,
 '기기': 58,
 '수업': 58,
 '과목': 58,
 '시작': 57,
 '엄마': 56,
 '내용': 56,
 '가지': 55,
 '월': 54,
 '필요': 54,
 '집': 54,
 '천재교육': 53,
 '7': 52,
 '5': 52,
 '추천': 51,
 '6': 49,
 '수준': 49,
 '관리': 48,
 '학습지': 47,
 '10': 47,
 '요즘': 47,
 '인': 46,
 '말': 46,
 '컨텐츠': 46,
 '고민': 45,
 '사용': 44,
 '이용': 44,
 '4': 44,
 '학원': 44,
 '강의': 44,
 '중요': 44,
 '게임': 44,
 '콘텐츠': 43,
 '세': 43,
 '초': 43,
 '설명': 43,
 '리': 43,
 '영상': 43,
 '터': 42,
 '집중': 42,
 '도움': 42,
 '내': 41,
 '확인': 41,
 '후기': 41,
 '방법': 41,
 '흥미': 41,
 '진도': 41,
 '동안': 40,
 '주': 40,
 '경우': 40,
 '글': 40,
 '활용': 40,
 '자동

In [ ]:
test_noun = []

for text in tqdm(df2.contents):
    # for noun in Kkma.nouns(text): #hannanum이 훨씬 빠르다 (Kkma : 3초에 1문장, hannanum : 1초에 5~10문장)
    for noun in hannanum.nouns(text):
        if noun not in stop_words:
            test_noun.append(noun)

test_count = Counter(test_noun)
test_top_nouns = dict(test_count.most_common(100))
test_top_nouns

100%|██████████| 100/100 [00:12<00:00,  7.77it/s]


{'학습': 563,
 '밀크티': 454,
 '공부': 334,
 '홈런': 247,
 '선생님': 239,
 '수': 210,
 '밀크T': 209,
 '선택': 187,
 '아이들': 181,
 '영어': 176,
 '초등': 173,
 '무료체험': 166,
 '생각': 164,
 '수학': 164,
 '아이스크림': 137,
 '천재교육': 132,
 '문제': 127,
 '학습기': 126,
 '체험': 125,
 '가능': 121,
 '콘텐츠': 117,
 '시작': 115,
 '시간': 114,
 '내용': 101,
 '신청': 100,
 '수업': 99,
 '학교': 97,
 '학습지': 95,
 '강의': 95,
 '교과서': 94,
 '엄마': 90,
 '사용': 89,
 '인강': 89,
 '컨텐츠': 82,
 '관리': 82,
 '설명': 80,
 '추천': 79,
 '혜택': 78,
 '경우': 77,
 '우리': 77,
 '지': 75,
 '진행': 74,
 '말': 72,
 '책': 70,
 '문제집': 69,
 '고민': 68,
 '과목': 65,
 '집': 65,
 '중': 63,
 '다양': 63,
 '영상': 63,
 '와이즈캠프': 62,
 '하나': 62,
 '태블릿': 62,
 '이용': 59,
 '코딩': 59,
 '국어': 59,
 '필요': 58,
 '활용': 58,
 '비교': 57,
 '제공': 57,
 '패드': 57,
 '이해': 56,
 '강': 56,
 '이야기': 56,
 '등': 55,
 '흥미': 54,
 '도움': 54,
 '학원': 53,
 '게임': 51,
 '이벤트': 50,
 '교재': 49,
 '초등인강': 49,
 '상품권': 48,
 '홈스쿨링': 48,
 '가격': 48,
 '정회원': 47,
 '걱정': 47,
 '적': 46,
 '스마트': 46,
 '중요': 46,
 '관심': 45,
 '한번': 45,
 '수준': 45,
 '엘리하': 44,
 '교육': 44,
 '장점': 

-------------------

In [ ]:
# test_noun2 = []

# for text in tqdm(df.contents):
#     # for noun in Kkma.nouns(text): #hannanum이 훨씬 빠르다 (Kkma : 3초에 1문장, hannanum : 1초에 5~10문장)
#     for noun in Kkma.nouns(text):
#         if noun not in stop_words:
#             test_noun2.append(noun)

# test_count2 = Counter(test_noun2)
# test_top_nouns2 = dict(test_count2.most_common(100))
# test_top_nouns2

In [ ]:
# test_noun = []

# for text in tqdm(df.contents):
#     # for noun in Kkma.nouns(text): #hannanum이 훨씬 빠르다 (Kkma : 3초에 1문장, hannanum : 1초에 5~10문장)
#     for noun in hannanum.nouns(text):
#         if noun not in stop_words:
#             test_noun.append(noun)

# test_count = Counter(test_noun)
# test_top_nouns = dict(test_count.most_common(100))
# test_top_nouns

In [ ]:
engel = []

for i in df2.contents:
    if '숨은천사' in i:
        engel.append(True)
    else:
        engel.append(False)

df2['engel'] = engel

In [ ]:
df2[df2['engel'] == True]

,contents,recommend,아이스크림,엘리하이,keyword_dict,engel


In [ ]:
dummy = df['contents'][302]

In [ ]:
A = '밀크티'
A in dummy

True

In [ ]:
df2

,contents,recommend,아이스크림,엘리하이,keyword_dict,engel
0,아이스크림홈런 천재교육밀크티 밀크T 밀크티 초등학습지 유아학습지 학습지선택 학습기 ...,False,True,True,{'밀크': [' 밀크T는 로그인이 홈런보다는복잡한 편인데요 로그인학습기를 켜면 밀...,False
1,우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T 무료 체험 이벤...,True,True,True,{'밀크': ['우리카드 9월 이벤트 밀크T 무료체험하고 혜택 100 받기■ 밀크T...,False
2,곧 초등학교에 입학하는 첫째에게시켜준없었어서방학기간에 초등학교 입학 준비를 시켜주기...,False,True,True,{'밀크': [' 이렇게 신청한 스마트 패드가5가지 교원 빨간펜 아이캔두 초등 천재...,False
3,안녕하세요 구미여신 입니다저는손으로 아이를 바르게키우고 싶다는 욕심이 있었어요학습 ...,False,True,True,{'밀크': [' 전업주부니까 그게 저의 직업이라고 생각했어요하지만 모든걸잘하고 싶...,False
4,앞에 올렸던 무료체험 비교 1을 계속 수정하려고 했는데 글이 너무 길어지는같아서다시...,True,True,True,{'밀크': [' 구분엘리하이7일윙크6일밀크T10일아이스크림 홈럼10일업체메가스터디...,False
...,...,...,...,...,...,...
95,교육에 그닥 관심없는 애미입니다방임은 아니고요 ㅎㅎ경험에서 우러나온 나름의 교육철학...,False,True,True,{'밀크': [' 밀크티는 2주였던것 같아요 암튼 약간의 혜택을 위해 일단 둘다 결...,False
96,안녕하세요해피블루입니다. 즐거운 주말 보내고 계신가요주말은 밍기적거리면서 낮잠도 자...,True,True,True,{'밀크': [' 밀크t only one 엘리하이 씽크빅스마트올 등등제가 광고로 들...,False
97,안녕하세요 룰디스입니다. 이제는 여름이라고 해도 손색이 없는 6월이네요. 얼마 전만...,True,True,True,{'밀크': [' 밀크T초등1등 교과서가 만든 AI맞춤학습m milkt co kr추...,False
98,초등학생 인강홈런 VS 밀크티 온라인학습 비교 무료체험 솔직 후기 홈런 오른 밀크티...,True,True,True,{'밀크': [' 그리고밀크T 학습기는 삼성전자 갤러시탭이라서 자체 제작 홈런의 태...,False


---------------------
``'03월 30일'``
-------------------

----------------
1. ```전처리 보수하기```
    - `현재 전처리 과정의 단계`
        1. 특정 단어가 들어간 문장 남기기(밀크티, 밀크 등의 단어가 들어가있는문장 -> 선생, 학습 등의 단어가 들어간 문장 __ 구조로 2번 처리)
            - 해당 조건을 모두 만족하는 더미데이터 발견
                - 방안1 : TF-IDF 를 이용하면 핵심 키워드 검출이 가능하므로 이용...? 근데 이러면 밀크티에 대한 본문이지만 밀크티가 빈출어가 아니라면 문제가 될듯...
                - 방안2 : 위의 전처리 단계에 한 단계를 더 추가한다... 어떤 단계를...?!
                
        2. 광고 여부확인
            - '추천' 등의 키워드가 들어가 있는지 여부로 확인

        3. 문장 구조 통일화
            - 정규식

-------------
2. `광고 이미지를 텍스트화 해서 광고글 분류`
    - `pytesseract` 패키지를 이용하여 광고글 확인
                - 밀크T 광고글은 특정 문구가 들어가있다. : '~~소정의 활동비를 받아 작성~'
                - 밀크T의 광고글이라는 표시로 문구가 이미지로 들어가 있는 경우가 있는데 이 때는 pytesseract를 활용하여 text화 시킨 후, 확인을 진행한다.
-------------

3. `공부 및 실습`
    - `CountVectorize`
        - 문장에서 단어의 출현 빈도에 따라서 표현하는 방법
        - 벡터화 해서 해당 단어들의 빈도수를 나타냄 
            (ex. '내가 그린 기린 그림은 긴 목 기린 그림 -> [[1 1 1 2 1]] {'내가': 4, '그린': 0, '기린': 3, '그림은': 2, '그림': 1})
        - CountVectorize를 사용하면 행렬로 표현 가능하다. -> 물론 dataframe화도 가능하다... 근데 만들어서 활용 방안이 딱히...?! 자연수로 표시된다...?

    - `TfidfVectorizer`
        - 문장에서 단순히 빈도수가 높은 단어가 핵심어가 아닌, 특정 문서에서만 집중적으로 등장할 때 해당 단어가 문서의 주제어(핵심어)라고 가정
            - 밀크T에 관한 문서들을 긁어왔을 때, 밀크T를 핵심어로 보는게 아닌 1번 문서에서는 '가격', 2번 문서에서는 '컨텐츠' 같은 방식인 듯 하다.
            - 편리하게 이용하려고 scikit-learn_Tfidfvectorizer를 사용한다.
                - 만약 핵심 단어를 찾지 못한다면...? 보수할 방안이 떠오르지는 않는다.
                    - 만약 가격에 대한 이야기가 주가되는 문서인데, '가격'이라는 단어보다 '비싸다', '저렴하다', '부담된다' 등의 단어들을 사용한다면...? 예외사항에 대한 대처가 힘들 수도 있을 듯 하다.
                    - 차라리 대처에 대한 방안보다는 stop_words 처럼 특정 단어들을 replace등의 함수를 사용하여 바꾼다면...?
                        - ex. '가격', '비용', '요금' 등의 단어를 한 단어로 묶어버리고 해당 단어가 나오는 문장에 나오는 '비싸다', '부담된다', '쎄다' 등의 단어를 묶어본다면 가능할지도 모르겠다.

        - 문장을 벡터화 및 가중치를 부여하여 나타낸다. CountVectorize의 가중치(idf)버전...?!
            - 머신러닝에 넣기 용이한 형태가 된다. 모델화가능...
                - 어떤 모델을 써야될지 고민해봐야 할 듯 하다.
                    (대충 생각나는거 써보기)
                    - 어떤 분류모델을 적용하여 리스트(키워드 해당 리스트 ex. price=['비싸다', '싸다' ...]의 긍/부정에 대한 빈도수를 독립변수로 삼고 필요한 종속변수(연장, 체험, 구입, 가입, 이탈, 취소)가 이뤄지는지 예측모델...?
                    - 어떤 회귀모델을 적용하여 특정 용언의 빈도수에 따른 만족도 점수 입력... 
                        - 내가 생각한 프로토타입 산출물에 가까운 생각이지만 개선점이나 어려움이 있을 듯 하다.
                            1. 점수 부여 기준 선정
                                1. 키워드의 빈도 수...-> 긍정적 키워드-부정적 키워드 = ? -> 이게 초기 발상이다.
                                1. 만족/불만족의 여부를 T/F로 나누어 머신러닝을 통해 만족여부를 예측해보고 만족글/전체글 로 점수부여
                            1. 점수 값 오류에 대한 대비
                                1. 점수 부여를 어떻게 할것인가... 아쉬웠다와 최악이다 는 부정적 언어지만 수준이 같은가...? 
                                1. 모델링 하는게 맞나...

-----------------------

## pytessearact
 - 이미지 텍스트화

In [ ]:
# !pip install pytesseract

In [ ]:
from PIL import Image
import pytesseract 

pytesseract.pytesseract.tesseract_cmd = 'C:/Users/user/AppData/Local/tessdata/'

img = Image.open('C:/Users/user/Documents/YE/milkt_support.png')
text = pytesseract.image_to_string(img,lang='kor')

print (text)

PermissionError: [WinError 5] 액세스가 거부되었습니다

-----------------------

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words='english').fit(test_noun)

print(tfidf.transform(test_noun).toarray())
print(tfidf.vocabulary_)
# value sort
print({k: v for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item: item[1])})

# key sort - 애초에 key값은 오름차순 정렬 되어있음.
# vector_key = list(tfidf.vocabulary_.keys())
# vector_key.sort()
# print({x: tfidf.vocabulary_[x] for x in vector_key})

In [ ]:
columns = []
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
  columns.append(k)

df = pd.DataFrame(tfidf.transform(test_noun).toarray(), columns=columns)
df